## Loading and cleaning the dataset

In [1]:
import re
import pandas as pd
from collections import Counter
import plotly.express as px

pd.set_option('display.max_colwidth', None)

df_recipes = pd.read_csv('recipes.csv')
df_recipes = df_recipes.drop_duplicates(subset='recipe_id', keep='first')

df_recipes['recipe_id'] = pd.to_numeric(df_recipes['recipe_id'], errors='coerce')
df_recipes.dropna(subset=['recipe_id'], inplace=True)

def remove_text_inside_parentheses(text):
    return re.sub(r'\([^)]*\)', '', text)

## Create column parsed_ingredients
df_recipes['parsed_ingredients'] = pd.Series(dtype='object')
for index, row in df_recipes.iterrows():
    ingredients_list = row['recipe_ingredients'].split('\n')
    cleaned_ingredient_names = set()
    for ingredient in ingredients_list:
        # Remove text inside parentheses
        ingredient = remove_text_inside_parentheses(ingredient)
        # Remove text after "-" and ":" then split by ',' and take the first part
        ingredient_name = ingredient.split('-')[0].split(':')[0].split(',')[0].lower().strip()
        cleaned_ingredient_names.add(ingredient_name)
    df_recipes.at[index, 'parsed_ingredients'] = cleaned_ingredient_names

## Create column num_unique_ingredients to count the number of ingredients for each recipe
df_recipes['num_ingredients'] = df_recipes['parsed_ingredients'].apply(lambda x: len([ingredient for ingredient in x if ingredient]))

df_recipes.describe()

,recipe_id,num_ingredients
count,18.000000,18.000000
mean,12436.555556,8.333333
std,155.453122,1.847096
min,12197.000000,5.000000
25%,12332.750000,7.000000
50%,12415.000000,8.000000
75%,12608.000000,9.750000
max,12638.000000,12.000000


## Ingredient Frequency Analysis

In [2]:
all_ingredients = []
most_used_ingredients_range = 25

for index, row in df_recipes.iterrows():
    for ingredient in row["parsed_ingredients"]:
        ingredient_name = ingredient.split(',')[0]
        all_ingredients.append(ingredient_name)

ingredient_counts = Counter(all_ingredients)
ingredient_df = pd.DataFrame.from_dict(ingredient_counts, orient='index', columns=['frequency'])
ingredient_df = ingredient_df.sort_values(by='frequency', ascending=False)

most_used_ingredients = ingredient_df.head(most_used_ingredients_range)
fig = px.bar(
    most_used_ingredients, 
    x=most_used_ingredients.index, 
    y='frequency',
    title=f'Top {most_used_ingredients_range} Most Common Ingredients',
    labels={'x': 'Ingredient', 'frequency': 'Frequency'
})

fig.update_layout(xaxis_tickangle=-45, xaxis_title='Ingredient', yaxis_title='Frequency', plot_bgcolor='white')
fig.show()

## Find the X recipes with the least amount of ingredients

In [3]:
sorted_recipes = df_recipes.sort_values(by='num_ingredients', ascending=True)
easiest_recipes_by_ingredients = sorted_recipes.iloc[:2]
easiest_recipes_by_ingredients

,recipe_id,recipe_name,recipe_description,recipe_nutriscore,recipe_ingredients,recipe_reproduction_steps,parsed_ingredients,num_ingredients
5,12578,Quiche jurassienne au morbier et poireaux,"Goûtez à l'authenticité avec notre quiche jurassienne : morbier fondant, poireaux tendres, une symphonie de saveurs à chaque bouchée. Irrésistible !",B,"Crème épaisse, 30 mL, ingredient-12578-2-12393\nMorbier AOP au lait cru de vache, 90 g, ingredient-12578-2-7667\nMâche, 50, ingredient-12578-2-11347\nPoireau, 2, ingredient-12578-2-10818\nPâte brisée ronde - Croustipate (230g), 1, ingredient-12578-2-11334","Step 1: Astuce : Pour éviter que votre pâte ne se réchauffe, sortez-la au dernier moment !\r\nPréchauffez votre four à 180°C en chaleur tournante !\r\nPendant ce temps, coupez et retirez la base abîmée des poireaux. Incisez-les en deux dans la longueur et rincez-les soigneusement. Émincez-les.\r\nDans une sauteuse, faites chauffer un filet d'huile d'olive à feu moyen à vif.\r\nFaites revenir les poireaux 10 min environ. Remuez de temps en temps. Salez, poivrez.\r\nEn parallèle, préparez la quiche.\nStep 2: Coupez le morbier en tranches.\r\nDans un bol, fouettez les oeufs et la crème. Salez, poivrez.\r\nDans un moule à tarte, déroulez la pâte brisée et piquez-la à l'aide d'une fourchette.\r\nAjoutez quelques tranches de morbier et les poireaux cuits.\r\nVersez l'appareil à oeuf et déposez le reste des tranches de morbier par-dessus.\r\nEnfournez 25 min environ.\r\nPendant ce temps, assaisonnez la mâche d'un filet d'huile d'olive et de vinaigre.\nStep 3: Servez sans attendre votre quiche jurassienne au morbier et poireaux accompagnée de la salade !","{poireau, mâche, morbier aop au lait cru de vache, pâte brisée ronde, crème épaisse}",5
16,12197,Velouté de butternut à la muscade et tartines au délice de Bourgogne à la truffe,Un velouté épicé de muscade et des tartines gourmandes au fromage bourguignon truffé ? Voici un plat réconfortant pour affronter l'hiver !,A,"Tranches de pain a la chataigne - Maison Salesse (x4), 4, ingredient-12197-2-16703\nCarotte, 2, ingredient-12197-2-8763\nCourge butternut, 300 g, ingredient-12197-2-12444\nCrème liquide, 100 mL, ingredient-12197-2-12392\nDélice de Bourgogne truffé, 60 g, ingredient-12197-2-2756\nMuscade moulue - Sur les quais (3g), 1, ingredient-12197-2-8265","Step 1: Astuce : Pour qu'elle soit plus facile à découper, passez la courge 5 min au micro-ondes à pleine puissance. Ensuite, manipulez-la avec un torchon pour ne pas vous brûler.\r\nÉpluchez et coupez le butternut en deux. Retirez les graines à l'aide d'une cuillère. Coupez-le en tranches puis taillez celles-ci en dés.\r\nEpluchez et coupez les carottes en morceaux.\r\nDans une grande casserole, déposez les légumes et versez de l'eau à hauteur de la garniture.\r\nPortez à ébullition et faites cuire 15 à 20 min jusqu'à ce que les légumes soient tendres.\r\nUne fois cuits, à l’aide d’un mixeur, mixez le tout jusqu'à obtenir une texture homogène.\r\nAstuce : ajoutez plus ou moins selon la texture souhaitée.\r\nAjoutez la crème, la muscade et mélangez bien. Salez, poivrez.\r\nGoûtez et rectifiez l'assaisonnement si nécessaire.\r\nEn parallèle, préparez les tartines.\nStep 2: Faites dorer les tranches de pain au grille-pain.\r\nPendant ce temps, coupez le délice de Bourgogne en tranches.\r\nUne fois le pain doré, déposez les tranches de fromage sur les tartines.\nStep 3: Servez sans attendre votre velouté de butternut à la muscade accompagné des tartines au délice de Bourgogne !","{muscade moulue, carotte, tranches de pain a la chataigne, crème liquide, courge butternut, délice de bourgogne truffé}",6


## Find recipes that contains certain aliments (fish, meat, etc..)

In [4]:
fish_keywords = ['poisson', 'saumon', 'merlu', 'cabillaud', 'truite', 'thon', 'bar', 'maquereau', 'dorade', 'sole', 'églefin', 'sardine', 'morue', 'turbot', 'lieu']
meat_keywords = ['viande', 'poulet', 'boeuf', 'volaille', 'steak', 'entrecôte', 'porc', 'cochon', 'dinde', 'saucisse', 'lapin']

def contains_keywords(ingredients_set, keywords):
    keywords = [keyword.lower() for keyword in keywords]
    return any(keyword in ingredient for ingredient in ingredients_set for keyword in keywords)

df_recipes['contains_fish'] = df_recipes['parsed_ingredients'].apply(lambda x: contains_keywords(x, fish_keywords))
df_recipes['contains_meat'] = df_recipes['parsed_ingredients'].apply(lambda x: contains_keywords(x, meat_keywords))

print("Recipes containing fish:", df_recipes[df_recipes['contains_fish']].shape[0])
print("Recipes containing meat:", df_recipes[df_recipes['contains_meat']].shape[0])

print(df_recipes[df_recipes['contains_fish'] == True][["recipe_id", "recipe_name"]].head(2))

Recipes containing fish: 1
Recipes containing meat: 5
   recipe_id                                 recipe_name
1      12618  Tartare de saumon fumé, betterave et pomme


TODO: try cosine similarity or other method ? Jaccard similarity ?

## Find similar recipes from a "Central" recipe

In [5]:
central_recipe_id = 12047
central_ingredients = df_recipes.loc[df_recipes['recipe_id'] == central_recipe_id, 'parsed_ingredients'].iloc[0]

# Define a function to calculate the number of shared ingredients
def calculate_shared_ingredients(ingredients_set, central_ingredients):
    return len(ingredients_set.intersection(central_ingredients))

# Apply this function across the DataFrame, excluding the central recipe itself
df_recipes['shared_ingredients_count'] = df_recipes.apply(lambda row: calculate_shared_ingredients(row['parsed_ingredients'], central_ingredients) if row['recipe_id'] != central_recipe_id else 0, axis=1)

# Sort the DataFrame based on 'shared_ingredients_count' to find the most similar recipes
similar_recipes = df_recipes[df_recipes['recipe_id'] != central_recipe_id].sort_values(by='shared_ingredients_count', ascending=False)

# Display the top similar recipes, excluding the central one
print(similar_recipes[['recipe_id', 'recipe_name', 'parsed_ingredients', 'shared_ingredients_count']].head())

IndexError: single positional indexer is out-of-bounds

In [ ]:
# Filter recipes that contain "concombre" in their parsed_ingredients
concombre_recipes = df_recipes[df_recipes['parsed_ingredients'].apply(lambda ingredients: 'concombre noa' in ingredients)]

# Display the filtered DataFrame to see the recipes containing "concombre"
print(concombre_recipes[['recipe_id', 'recipe_name', 'parsed_ingredients']])

     recipe_id  \
96     10980.0   
115    10905.0   
117    10698.0   
126    10780.0   
127    10708.0   
128    10696.0   
141    10601.0   
145    10592.0   
149    10563.0   
150    10693.0   
152    10558.0   
159    10466.0   
164    10407.0   
167    10369.0   
169    10522.0   
175     9955.0   
183    10306.0   
204     9917.0   
211     9599.0   
219     9728.0   
226     9708.0   
229     9098.0   
231     9158.0   
233     9205.0   
234     9149.0   
244     9209.0   
249     8975.0   
366     7440.0   
374     7432.0   

                                                                                recipe_name  \
96                     Sabich de patate douce au crémeux d'avocat et pickles d'oignon rouge   
115                               Salade de pastèque au maïs grillé et vinaigrette pimentée   
117                  Salade de concombre/courgette au piment chipotle, yaourt grec et aneth   
126                                           Salade de pastèque grillée, féta 